# para achar o race.csv e para ver as colunas e tipo delas

In [0]:
dfLeitura = spark.read.format("delta").load("dbfs:/mnt/formula1/bronze/race_raw.delta")

##para renomeiar as duas colunas

In [0]:
dfLeitura = dfLeitura.withColumnRenamed("raceId", "Race_id") \
                     .withColumnRenamed("circuitId", "Circuit_id")

## para deletar url

In [0]:
dfLeitura = dfLeitura.drop("url")

## para criar uma nova coluna para receber o tempo alterado

In [0]:
from pyspark.sql.functions import lit, concat, to_timestamp, date_format
dfLeitura = dfLeitura.withColumn("coluna_do_tempo", date_format(to_timestamp(concat(dfLeitura["date"], lit(" "), dfLeitura["time"])), "yyyy-MM-dd HH:mm:ss"))

In [0]:
%python
dfLeitura.write.format('delta').mode('overwrite').save('dbfs:/mnt/formula1/prata/race_raw.delta')